In [1]:
import pickle 
import numpy as np
import pandas as pd
import statistics
from PIL import Image
import os 
import random
import time
import matplotlib.pyplot as plt
import netaddr

In [2]:
import dgl
import torch
import torch.nn as nn
import dgl.function as fn
import dgl.nn as dglnn
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

In [3]:
#pip install netaddr

In [4]:
#directory should change based on the device, for laptop it should be users//jalal for office desktop it is users//jghadermazi

In [5]:
wed_data = pd.read_pickle(r'C:\\Users\\Jalal\\Box\\GAN Project\\Sample data\\Wednesday_Packet_Level_Data.pkl')

In [6]:
df1 = wed_data.copy()

In [7]:
del wed_data

In [8]:
df1[['Payload']] = df1[['Payload']].replace('', '00')

In [9]:
df1

,Epoch Time,Source IP,Dst IP,Source Port,Dst Port,Stream Index,Sequence Number,Eth Header,IP Header,TCP Header,Payload,Direction,Binary Label,Class Label
0,1499258820.014743000,23.60.139.27,192.168.10.17,80,47757,12035,1,0023ae9b956700c1b114eb310800,45000034a7b6400037062efd173c8b1bc0a80a11,0050ba8da529957b5d3f78e3801003ab15920000010108...,00,fwd,1.0,1.0
1,1499258820.039091000,23.60.139.27,192.168.10.17,80,47757,12035,1,0023ae9b956700c1b114eb310800,45000722a7b740003706280e173c8b1bc0a80a11,0050ba8da529957b5d3f78e3801803ab74250000010108...,485454502f312e3120323030204f4b0d0a536572766572...,bwd,1.0,1.0
2,1499258820.039116000,192.168.10.17,23.60.139.27,47757,80,12035,435,00c1b114eb310023ae9b95670800,45000034fcd740004006d0dbc0a80a11173c8b1b,ba8d00505d3f78e3a5299c698010010011280000010108...,00,fwd,1.0,1.0
3,1499258820.082489000,192.168.10.12,208.185.50.90,51774,443,11808,4580,00c1b114eb31b8ac6f3604e30800,4500002804414000400668c7c0a80a0cd0b9325a,ca3e01bb4c69a4a67cf3967050109eb472ea0000,000000000000,fwd,1.0,1.0
4,1499258820.084153000,192.168.10.17,23.208.97.23,37115,443,12034,349,00c1b114eb310023ae9b95670800,4500017a021640004006f3c7c0a80a1117d06117,90fb01bb96268b0ed08217e7801801356d9e0000010108...,1703030141000000000000000174b7a44755da49b50c35...,fwd,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11334902,1499276130.701758000,172.16.0.1,192.168.10.51,443,53199,243533,0,b8ac6f360ba800c1b114eb310800,4500003cc45240003f06007dac100001c0a80a33,01bbcfcf167451881e1ff3d6a012fe884e0a0000020405...,00,bwd,0.0,0.0
11334903,1499276130.702180000,172.16.0.1,192.168.10.51,443,53199,243533,1,b8ac6f360ba800c1b114eb310800,45000034034d40003f06c18aac100001c0a80a33,01bbcfcf167451891e1ff5db8010000879580000010108...,00,bwd,0.0,0.0
11334904,1499276130.702964000,172.16.0.1,192.168.10.51,443,53199,243533,1,b8ac6f360ba800c1b114eb310800,450000bd034e40003f06c100ac100001c0a80a33,01bbcfcf167451891e1ff5db801800084ed60000010108...,16030300510200004d0303587c9e338fe3870b8fd3c17c...,bwd,0.0,0.0
11334905,1499276130.710822000,172.16.0.1,192.168.10.51,443,53199,243533,138,b8ac6f360ba800c1b114eb310800,45000034034f40003f06c188ac100001c0a80a33,01bbcfcf167452121e1ffa1f80100008748a0000010108...,00,bwd,0.0,0.0


In [10]:
#benign    0
#dos slowloris    1
#dos slowhttptest   2
#dos hulk     3
#dos goldeneye    4
#heartbleed    5
#ftp patator    6
#ssh patator    7
#webattack bruteforce    8
#webattack xss     9
#webattack sql injection 10
#infiltration     11
#botnet     12
#portscan   13
#ddos     14

In [11]:
df1['Class Label'].value_counts()

0.0    8845368
3.0    2246818
4.0     106177
5.0      49296
1.0      47586
2.0      39662
Name: Class Label, dtype: int64

In [12]:
df = df1[df1['Class Label']== 2]

In [13]:
df = df.reset_index(inplace = False)

In [14]:
df

,index,Epoch Time,Source IP,Dst IP,Source Port,Dst Port,Stream Index,Sequence Number,Eth Header,IP Header,TCP Header,Payload,Direction,Binary Label,Class Label
0,47586,1499259207.042142000,205.185.216.42,192.168.10.51,443,48338,14644,157223,b8ac6f360ba800c1b114eb310800,450005dcbdd9400039060d83cdb9d82ac0a80a33,01bbbcd28f889240889c36d68010001f96d40000010108...,149a41c9cac64a91e9122e1cab38162362110c2ab150fd...,fwd,1.0,2.0
1,47587,1499259207.042213000,192.168.10.51,205.185.216.42,48338,443,14644,649,00c1b114eb31b8ac6f360ba80800,4500003465f1400040066413c0a80a33cdb9d82a,bcd201bb889c36d68f88a3388010080b93a10000010108...,00,fwd,1.0,2.0
2,47588,1499259207.042334000,205.185.216.42,192.168.10.51,443,48338,14644,158671,b8ac6f360ba800c1b114eb310800,45000b84bdda4000390607dacdb9d82ac0a80a33,01bbbcd28f8897e8889c36d68010001f7c360000010108...,27c2bf56ed7a9006c96c04308197a2683cb16bff045d3f...,bwd,1.0,2.0
3,47589,1499259207.042525000,205.185.216.42,192.168.10.51,443,48338,14644,161567,b8ac6f360ba800c1b114eb310800,450005dcbddc400039060d80cdb9d82ac0a80a33,01bbbcd28f88a338889c36d68010001f68430000010108...,657dfa8e25142135593add180356a88fa823f54f1d9c96...,bwd,1.0,2.0
4,47590,1499259207.042709000,192.168.10.51,205.185.216.42,48338,443,14644,649,00c1b114eb31b8ac6f360ba80800,4500003465f2400040066412c0a80a33cdb9d82a,bcd201bb889c36d68f88b9d8801008667ca60000010108...,00,fwd,1.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39657,87243,1499259296.846679000,172.16.0.1,192.168.10.50,56688,80,15722,1,0019b90a69f100c1b114eb310800,45000034a06840003e062570ac100001c0a80a32,dd70005068b8f63c6bbaf7c5801000e5bb120000010108...,00,fwd,1.0,2.0
39658,87244,1499259296.847061000,172.16.0.1,192.168.10.50,56688,80,15722,1,0019b90a69f100c1b114eb310800,4500011ba06940003e062488ac100001c0a80a32,dd70005068b8f63c6bbaf7c5801800e59b690000010108...,474554202f20485454502f312e310d0a486f73743a2032...,fwd,1.0,2.0
39659,87245,1499259296.847138000,172.16.0.1,192.168.10.50,56690,80,15723,0,0019b90a69f100c1b114eb310800,4500003c2e3240003e06979eac100001c0a80a32,dd72005071fb786e00000000a00272103f220000020405...,00,fwd,1.0,2.0
39660,87246,1499259296.851569000,172.16.0.1,192.168.10.50,56672,80,15713,0,0019b90a69f100c1b114eb310800,4500003ca82a40003e061da6ac100001c0a80a32,dd6000505a9ffb4700000000a0027210d3b40000020405...,00,fwd,1.0,2.0


In [15]:
ip_header_lst = []

for i in range(len(df)):
    x = df['IP Header'][i]
    byte_array = bytes.fromhex(x)
    byte_lst = list(byte_array)
    if (len(byte_lst) < 20):
            output = np.pad(byte_lst, (0, 20-len(byte_lst)), 'constant')
    else:
            output = np.array(byte_lst[0:20].copy())
            
    output = np.delete(output, [0, 1, 9, 12, 13, 14, 15, 16, 17, 18, 19])
    
    output = np.abs(output.astype(float))/255
    ip_header_lst.append(output)
    
df['IP Header Array'] = ip_header_lst

In [16]:
tcp_header_lst = []

for i in range(len(df)):
    x = df['TCP Header'][i]
    byte_array = bytes.fromhex(x)
    byte_lst = list(byte_array)
    if (len(byte_lst) < 20):
            output = np.pad(byte_lst, (0, 20-len(byte_lst)), 'constant')
    else:
            output = np.array(byte_lst[0:20].copy())
            
    output = np.delete(output, [0, 1, 2, 3])
    output = np.abs(output.astype(float))/255
    
    tcp_header_lst.append(output)
    
df['TCP Header Array'] = tcp_header_lst

In [17]:
df['pkt_header_values'] = df.apply(lambda x : (np.append(x['IP Header Array'], x['TCP Header Array'])), axis=1)

In [18]:
new_cols = ['Source IP', 'Dst IP', 'Source Port', 'Dst Port','Epoch Time','Stream Index', 'Sequence Number', 'Direction', 'pkt_header_values']
df = df[new_cols]

In [19]:
# 70% of benign data for train, 30% for test benign

In [20]:
#random.seed(0)
#mlist = random.sample(streams_lst, 61492)

In [21]:
#len(imlist)

In [22]:
#df_train = df.loc[df['Stream Index'].isin(imlist)]
#df_train = df_train.reset_index(inplace = False)

In [23]:
#df_test = df.loc[~df['Stream Index'].isin(imlist)]
#df_test = df_test.reset_index(inplace = False)

In [24]:
#streams_lst1 = list(df_test['Stream Index'].unique())

In [25]:
#len(streams_lst1)

In [26]:
df

,Source IP,Dst IP,Source Port,Dst Port,Epoch Time,Stream Index,Sequence Number,Direction,pkt_header_values
0,205.185.216.42,192.168.10.51,443,48338,1499259207.042142000,14644,157223,fwd,"[0.0196078431372549, 0.8627450980392157, 0.741..."
1,192.168.10.51,205.185.216.42,48338,443,1499259207.042213000,14644,649,fwd,"[0.0, 0.20392156862745098, 0.396078431372549, ..."
2,205.185.216.42,192.168.10.51,443,48338,1499259207.042334000,14644,158671,bwd,"[0.043137254901960784, 0.5176470588235295, 0.7..."
3,205.185.216.42,192.168.10.51,443,48338,1499259207.042525000,14644,161567,bwd,"[0.0196078431372549, 0.8627450980392157, 0.741..."
4,192.168.10.51,205.185.216.42,48338,443,1499259207.042709000,14644,649,fwd,"[0.0, 0.20392156862745098, 0.396078431372549, ..."
...,...,...,...,...,...,...,...,...,...
39657,172.16.0.1,192.168.10.50,56688,80,1499259296.846679000,15722,1,fwd,"[0.0, 0.20392156862745098, 0.6274509803921569,..."
39658,172.16.0.1,192.168.10.50,56688,80,1499259296.847061000,15722,1,fwd,"[0.00392156862745098, 0.10588235294117647, 0.6..."
39659,172.16.0.1,192.168.10.50,56690,80,1499259296.847138000,15723,0,fwd,"[0.0, 0.23529411764705882, 0.1803921568627451,..."
39660,172.16.0.1,192.168.10.50,56672,80,1499259296.851569000,15713,0,fwd,"[0.0, 0.23529411764705882, 0.6588235294117647,..."


In [27]:
streams_lst = list(df['Stream Index'].unique())
print('Total number of flows in class:', len(streams_lst))    
random.shuffle(streams_lst)

# Directory path where to save the CSV files
directory_path = 'Wednesday_slowlhttptest_all/'

for PKT in range(2, 16):
    
    file_name = f'wednesday_slowlhttptest_{PKT}pkt.csv'
    file_path = directory_path + file_name
    

    data = []
    num_pkt = PKT
    pkt_val_pad = [0] * 25


    for j in range(len(streams_lst)):

        df_stream = df[df['Stream Index'] == streams_lst[j]]


        df_stream = df_stream.sort_values(by = 'Epoch Time')
        df_stream = df_stream.reset_index(drop=True)


        if len(df_stream) >= num_pkt:

            src_node = (int(netaddr.IPAddress(df_stream['Source IP'][0])), int(df_stream['Source Port'][0]))
            dst_node = (int(netaddr.IPAddress(df_stream['Dst IP'][0])), int(df_stream['Dst Port'][0]))
            stream_no = df_stream['Stream Index'][0]

            pkt_val_seq = []

            for k in range(0, num_pkt):

                pkt_val = []
                if df_stream['Direction'][k] == 'fwd':
                    pkt_val = [*df_stream['pkt_header_values'][k], *pkt_val_pad]
                else:
                    pkt_val = [*pkt_val_pad, *df_stream['pkt_header_values'][k]]

                pkt_val_seq.extend(pkt_val)

            data.append([src_node, dst_node, stream_no, pkt_val_seq])

    df2 = pd.DataFrame(data, columns= ['src_node', 'dst_node', 'stream_number', 'pkt_vals'])
    
    df2.to_csv(file_path, index=False)

Total number of flows in class: 1586
